<a href="https://colab.research.google.com/github/Mary7Magdalene/MLL/blob/main/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Types of movie recommendation systems


*  Content based recommendation systems- recommends movies to a user based on the movies they have watched before
*  Popularity based recommendation systems - recommends movies which have been popular to the user.


*  colaborative recommendation systems- groups people based on their watching pattern.





In [3]:
#importing dependancies
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#data collection and


In [10]:
#loading the movies data from a csv into a pandas datadframe
#loading the movies data from a csv into a pandas datadframe
# Adding on_bad_lines=False to skip problematic rows
# and engine='python' to use the Python parser instead of the C parser
movies_data = pd.read_csv('/content/movies.csv',
                          on_bad_lines = 'skip',
                          engine = 'python')

In [12]:
#printing the first five rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [14]:
movies_data.shape

(533, 24)

In [15]:
#selecting the relevant features for recommendation
selected_features = ['genres', 'keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [16]:
#replacing the missing value with a null value
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [18]:
#combining all the file of the selected feature
combined_features = movies_data['genres'] +''+['keywords']+''+['tagline']+''+['cast']+''+['director']

In [19]:
print(combined_features)

0      Action Adventure Fantasy Science Fictionkeywor...
1      Adventure Fantasy Actionkeywordstaglinecastdir...
2      Action Adventure Crimekeywordstaglinecastdirector
3      Action Crime Drama Thrillerkeywordstaglinecast...
4      Action Adventure Science Fictionkeywordstaglin...
                             ...                        
528    Drama Action History Thrillerkeywordstaglineca...
529          Action Drama Warkeywordstaglinecastdirector
530    Action Comedy Thriller Romancekeywordstaglinec...
531    Comedy Action Adventurekeywordstaglinecastdire...
532                    Comedykeywordstaglinecastdirector
Name: genres, Length: 533, dtype: object


In [20]:
#converting the data into numerical value
vectorizer = TfidfVectorizer()


In [21]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [22]:
print(feature_vectors)

  (0, 0)	0.3311328569296591
  (0, 2)	0.34995322139271795
  (0, 15)	0.5358818089571218
  (0, 29)	0.43993766326167955
  (0, 18)	0.5358818089571218
  (1, 2)	0.36149353100986076
  (1, 15)	0.5535534336644131
  (1, 1)	0.7502673011109845
  (2, 0)	0.34322342308394566
  (2, 2)	0.362730970521531
  (2, 9)	0.8663855520904347
  (3, 0)	0.30596869874510213
  (3, 8)	0.6553544116788055
  (3, 11)	0.48342479484710527
  (3, 31)	0.4931472581373539
  (4, 0)	0.3922013483302794
  (4, 2)	0.41449261953459526
  (4, 29)	0.5210722557476555
  (4, 18)	0.6347107018235236
  (5, 0)	0.3649218838290571
  (5, 15)	0.5905635612472504
  (5, 3)	0.7197685036382302
  (6, 4)	0.6738790258555567
  (6, 14)	0.7388417005773064
  (7, 0)	0.3922013483302794
  :	:
  (524, 11)	0.41354882392683834
  (524, 6)	0.3829067755737166
  (524, 20)	0.826050707552792
  (525, 0)	0.3368206028420484
  (525, 4)	0.5406846338065706
  (525, 14)	0.5928072235376143
  (525, 6)	0.4927388801263796
  (526, 12)	1.0
  (527, 9)	0.8652036256483657
  (527, 6)	0.501420

In [25]:
similarity = cosine_similarity(feature_vectors)

In [26]:
print(similarity)

[[1.         0.42314504 0.24059142 ... 0.09730212 0.12488525 0.        ]
 [0.42314504 1.         0.1311249  ... 0.         0.         0.        ]
 [0.24059142 0.1311249  1.         ... 0.10085489 0.12944515 0.        ]
 ...
 [0.09730212 0.         0.10085489 ... 1.         0.34799575 0.        ]
 [0.12488525 0.         0.12944515 ... 0.34799575 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [27]:
similarity.shape

(533, 533)

In [39]:
movie_name =  input('Enter your favouritr movie name:')

Enter your favouritr movie name:YOU


In [29]:
#creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [31]:
#finding a close match of the movie name given by user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Twister', 'The Pink Panther', 'The Other Guys']


In [32]:
close_match = find_close_match[0]
print(close_match)

Twister


In [33]:
#finding the index of the nmovie
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

383


In [40]:
#printing the list of similar movies
similarity_score = list( enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.266316752740378), (1, 0.14514547847955192), (2, 0.2760407056783563), (3, 0.11624445921615728), (4, 0.31543166835269887), (5, 0.13864211344438765), (6, 0.0), (7, 0.31543166835269887), (8, 0.16232612120477624), (9, 0.33570654979030656), (10, 0.266316752740378), (11, 0.23436533231268214), (12, 0.14514547847955192), (13, 0.21806196350109036), (14, 0.266316752740378), (15, 0.15323578936362545), (16, 0.11964421066536096), (17, 0.33570654979030656), (18, 0.13850457963491952), (19, 0.33570654979030656), (20, 0.33570654979030656), (21, 0.17180128590244817), (22, 0.19491497435236654), (23, 0.19491497435236654), (24, 0.14619822000843027), (25, 0.0), (26, 0.31543166835269887), (27, 0.2570629203836268), (28, 0.25551055459232336), (29, 0.3703998688343527), (30, 0.33570654979030656), (31, 0.31543166835269887), (32, 0.0), (33, 0.25551055459232336), (34, 0.0), (35, 0.11964421066536096), (36, 0.11964421066536096), (37, 0.16232612120477624), (38, 0.33570654979030656), (39, 0.31543166835269887), (4

In [41]:
#sorting the movies basedon their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(237, 1.0), (383, 1.0), (475, 1.0), (384, 0.9250182319421086), (192, 0.8333327895445499), (214, 0.8333327895445499), (250, 0.8333327895445499), (264, 0.8333327895445499), (357, 0.8333327895445499), (448, 0.8333327895445499), (526, 0.8333327895445499), (80, 0.7703678214184296), (103, 0.7699293945851388), (408, 0.7319298587133577), (434, 0.6932832848268236), (521, 0.6932832848268236), (119, 0.6846402872229597), (60, 0.6724790531442054), (327, 0.6705698882391982), (257, 0.6655428660114522), (491, 0.5744018596802946), (494, 0.5690736745645623), (112, 0.5417373396122835), (463, 0.5241151617375032), (377, 0.5131857779002125), (444, 0.5131857779002125), (239, 0.5013635521649374), (440, 0.48331581853847166), (280, 0.4552984595215918), (326, 0.43433600447493936), (354, 0.418698190606397), (29, 0.3703998688343527), (134, 0.3703998688343527), (139, 0.3703998688343527), (147, 0.3703998688343527), (170, 0.3703998688343527), (213, 0.3703998688343527), (240, 0.3703998688343527), (245, 0.370399868834

In [38]:
#print the name of similar movies based on the index
print('Movies suggested for you: \n')
i = 1
for movie in sorted_similar_movies:
 index = movie[0]
 title_from_index = movies_data[movies_data.index == index]['title'].values[0]
 if (i<30):
  print(i, '.',title_from_index)
  i+=1

Movies suggested for you: 

1 . The Huntsman: Winter's War
2 . Twister
3 . The Edge
4 . Cast Away
5 . Australia
6 . The Perfect Storm
7 . The Aviator
8 . Ali
9 . Ben-Hur
10 . Cold Mountain
11 . Dreamgirls
12 . Snow White and the Huntsman
13 . The Sorcerer's Apprentice
14 . Collateral Damage
15 . The Longest Yard
16 . The Terminal
17 . Batman Begins
18 . A Christmas Carol
19 . The Lovely Bones
20 . Real Steel
21 . Pompeii
22 . The Lion King
23 . Alexander
24 . Déjà Vu
25 . The Devil's Own
26 . Road to Perdition
27 . Gravity
28 . Four Christmases
29 . Public Enemies


MOVIE RECOMMENDATION SYSTEM

In [43]:
movie_name =  input('Enter your favouritr movie name:')
#creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)
#finding a close match of the movie name given by user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)
close_match = find_close_match[0]
print(close_match)
#finding the index of the nmovie
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)
#printing the list of similar movies
similarity_score = list( enumerate(similarity[index_of_the_movie]))
print(similarity_score)
#sorting the movies basedon their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)
#print the name of similar movies based on the index
print('Movies suggested for you: \n')
i = 1
for movie in sorted_similar_movies:
 index = movie[0]
 title_from_index = movies_data[movies_data.index == index]['title'].values[0]
 if (i<30):
  print(i, '.',title_from_index)
  i+=1

Enter your favouritr movie name:danger
['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2'